### Fetch spot yield for different currencies "https://www.tradingview.com/markets/bonds/prices-all/"

In [87]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import re
import os
import warnings
warnings.filterwarnings("ignore")
import sys

sys.path.append(os.getcwd())
from utils import *

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import lxml
import plotly.graph_objects as go


&rarr; for chrome


In [88]:
# def PullData(url):
#     driver = webdriver.Chrome(executable_path=r"C:\Program Files (x86)\Google\Chrome\chromedriver.exe")
#     driver.get(url)
#     time.sleep(2)
#     scroll_pause_time = 1
#     screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
#     i = 1
#     while True:
#         driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
#         i += 1
#         time.sleep(scroll_pause_time)
#         scroll_height = driver.execute_script("return document.body.scrollHeight;")  
#         if (screen_height) * i > scroll_height:
#             break
#     while True:
#         try:
#             loadMoreButton = driver.find_element_by_class_name("tv-load-more__btn")
#             loadMoreButton.click()
#             time.sleep(5)
#         except:
#             break
#     zero_rates = pd.read_html(driver.page_source)[1]
#     driver.close()
#     zero_rates = zero_rates.iloc[:,[0,2,3]].replace('—',np.nan).dropna()
#     zero_rates.columns = ["Ticker","Maturity", "Yield"]
#     zero_rates["Maturity"] = pd.to_datetime(zero_rates["Maturity"])
#     zero_rates[["Yield"]] = zero_rates[["Yield"]].replace('−','-', regex=True)
#     zero_rates.iloc[:,0] = zero_rates.iloc[:,0].apply(lambda x: x[0:5])
#     zero_rates["Yield"] = pd.to_numeric(zero_rates["Yield"])
#     return zero_rates

 safaridriver --enable    

In [89]:
def PullData(url):
  
    driver = webdriver.Safari()
    driver.get(url)
    time.sleep(2) 

   
    scroll_pause_time = 1
    screen_height = driver.execute_script("return window.screen.height;")  # Get screen height
    i = 1

    while True:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break  # Stop scrolling when bottom is reached

    # Click "Load More" button (if available)
    while True:
        try:
            load_more_button = driver.find_element("class name", "tv-load-more__btn")
            load_more_button.click()
            time.sleep(5)  # Wait for content to load
        except NoSuchElementException:
            break  # No more "Load More" button

    # Extract table data

    tables = pd.read_html(driver.page_source)
    # for i, table in enumerate(tables):
    #     print(f"Table {i}:")
    #     print(table.head())  

    # zero_rates = pd.read_html(driver.page_source)[1]
    driver.quit()  # close the browser
    
    zero_rates = tables[1]
    print("Columns in extracted table:", zero_rates.columns)
    zero_rates.columns = ["Symbol", "Coupon", "Yield %", "Maturity date", "Term to maturity", "Price", "Change %", "Change"]
    zero_rates = zero_rates[["Symbol", "Maturity date", "Yield %"]]
    zero_rates.columns = ["Ticker", "Maturity", "Yield"]

    zero_rates["Maturity"] = pd.to_datetime(zero_rates["Maturity"], errors="coerce")

 
    zero_rates["Yield"] = zero_rates["Yield"].str.replace('%', '', regex=False) 
    zero_rates["Yield"] = pd.to_numeric(zero_rates["Yield"], errors="coerce")  


    zero_rates["Ticker"] = zero_rates["Ticker"].str[:5]
    return zero_rates


In [ ]:
ticker_meta = pd.read_csv('data/ticker.csv')
url = "https://www.tradingview.com/markets/bonds/prices-all/"
zero_rates = PullData(url)
zero_rates.to_csv('data/zero_rates.csv', index=False)

Columns in extracted table: Index(['Symbol', 'Coupon', 'Yield %', 'Maturity date', 'Term to maturity',
       'Price', 'Change %', 'Change'],
      dtype='object')


In [92]:
zero_rates

,Ticker,Maturity,Yield
0,US01M,2025-03-25,4.321
1,US02M,2025-04-22,4.322
2,US03M,2025-05-22,4.307
3,US04M,2025-06-24,4.321
4,US06M,2025-08-21,4.338
...,...,...,...
95,ES09Y,2034-04-30,3.044
96,ES10Y,2035-04-30,3.195
97,ES15Y,2039-07-30,3.434
98,ES20Y,2043-07-30,3.607


In [105]:
zero_curve = zero_rates

In [106]:
zero_curve["T"] = zero_rates["Maturity"].apply(lambda x: ((x - dt.datetime.now()).days)/365)

In [107]:
zero_curve["Df"] = 1/(1+zero_rates["Yield"]/100)**zero_rates["T"]

In [108]:
# zero_curve = pd.merge(zero_curve, ticker_meta).dropna()

In [109]:
zero_curve

,Ticker,Maturity,Yield,T,Df,Tenor
0,US01M,2025-03-25,4.321,0.079452,0.996645,0.083333
1,US02M,2025-04-22,4.322,0.156164,0.993414,0.166667
2,US03M,2025-05-22,4.307,0.238356,0.989999,0.250000
3,US04M,2025-06-24,4.321,0.328767,0.986189,0.333333
4,US06M,2025-08-21,4.338,0.487671,0.979504,0.500000
...,...,...,...,...,...,...
95,ES09Y,2034-04-30,3.044,9.183562,0.759286,NaN
96,ES10Y,2035-04-30,3.195,10.183562,0.725949,NaN
97,ES15Y,2039-07-30,3.434,14.435616,0.614223,NaN
98,ES20Y,2043-07-30,3.607,18.438356,0.520296,NaN


In [112]:
def extract_tenor(ticker):
    match = re.search(r'(\d+)([MY])$', ticker)  # Extract the number and unit (M/Y)
    if match:
        value, unit = match.groups()
        value = int(value)
        if unit == 'M':  # Convert months to years
            return value / 12
        elif unit == 'Y':  # Years remain the same
            return value
    return None  # Return None if ticker format is unknown




In [113]:
zero_curve["Tenor"] = zero_curve["Ticker"].apply(extract_tenor)

In [ ]:
# zero_curve.to_csv("zero_curve.csv", index=False)

* T : The remaining time to maturity in years, calculated from today’s date.
* Df discount_factor : The present value of $1 received at maturity. Lower values indicate higher discounting (i.e., higher interest rates)
* yields are annualised
* Tenor: bond term in years

In [115]:
zero_curve

,Ticker,Maturity,Yield,T,Df,Tenor
0,US01M,2025-03-25,4.321,0.079452,0.996645,0.083333
1,US02M,2025-04-22,4.322,0.156164,0.993414,0.166667
2,US03M,2025-05-22,4.307,0.238356,0.989999,0.250000
3,US04M,2025-06-24,4.321,0.328767,0.986189,0.333333
4,US06M,2025-08-21,4.338,0.487671,0.979504,0.500000
...,...,...,...,...,...,...
95,ES09Y,2034-04-30,3.044,9.183562,0.759286,9.000000
96,ES10Y,2035-04-30,3.195,10.183562,0.725949,10.000000
97,ES15Y,2039-07-30,3.434,14.435616,0.614223,15.000000
98,ES20Y,2043-07-30,3.607,18.438356,0.520296,20.000000


In [116]:
# fwd_rate = Instantaneous_Forward_Rate(StDate, Tenor, curr, data)
# print("Forward Rate:", fwd_rate)

In [117]:
# Instantaneous_Forward_Rate(dt.date.today() + dt.timedelta(days=1800), 0.25, "USD", data)

In [118]:
# spot = zero_rates[zero_rates["Currency"]==curr]["Yield"]
# fwd3m, fwd6m, fwd1y = [], [], []
# spottimelist = zero_rates[zero_rates["Currency"]==curr]["Tenor"]
# timelist = np.linspace(0,20,80)
# for i in timelist:
#     fwd3m.append(Instantaneous_Forward_Rate(dt.date.today() + dt.timedelta(days=i*365), 0.25, "USD", data))
#     fwd6m.append(Instantaneous_Forward_Rate(dt.date.today() + dt.timedelta(days=i*365), 0.5, "USD", data))
#     fwd1y.append(Instantaneous_Forward_Rate(dt.date.today() + dt.timedelta(days=i*365), 1, "USD", data))

In [119]:
# fig = go.Figure()


# fig.add_trace(go.Scatter(x=spottimelist, y=spot, mode='lines', name='Spot Rate'))

# fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd3m, 100), mode='lines', name='3M Fwd'))
# fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd6m, 100), mode='lines', name='6M Fwd'))
# fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd1y, 100), mode='lines', name='1Y Fwd'))

# fig.update_layout(
#     title="Forward Rate Curves",
#     xaxis_title="Time to Maturity (Years)",
#     yaxis_title="Forward Rate (%)",
#     template="plotly_dark", 
#     width=900,
#     height=500
# )

# fig.show()

fine-tune the forward rate model(smothing the spike ?!) ...


In [ ]:
pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/all/202202?type=daily_treasury_yield_curve&field_tdr_date_value_month=202202&page&_format=csv')
